In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202540.csv")

In [3]:
ema[(ema["store_name"]=="SUCURSAL BUCARAMANGA") & (ema["product_ref"]=="BAB02013125")].tail(5)

,Unnamed: 0,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,lost_sales,producto_costo_unitario
18730,18730,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-10-21 00:00:00,43,0.0,0.0,0.0,0.0,0.0,0.0,227670.52
18731,18731,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-10-28 00:00:00,44,0.0,0.0,0.0,0.0,0.0,0.0,227670.52
18732,18732,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-11-04 00:00:00,45,0.0,0.0,0.0,0.0,0.0,0.0,227670.52
18733,18733,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-11-11 00:00:00,46,0.0,0.0,0.0,0.0,0.0,0.0,227670.52
18734,18734,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-11-18 00:00:00,47,0.0,0.0,0.0,0.0,0.0,0.0,227670.52


In [4]:
import numpy as np

ema = ema.copy()

# 1) Calcular venta_costo_ema (base)
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# 1b) Forzar cero para filas sin EMA (evita contaminación)
ema.loc[ema["EMA"].fillna(0) == 0, "venta_costo_ema"] = 0

# 2) Totales por sucursal (con los valores ya ajustados)
ema["venta_costo_tot"] = ema.groupby("store_name")["venta_costo_ema"].transform("sum")
ema["total_ema"] = ema.groupby("store_name")["EMA"].transform("sum")

# 3) Proporciones dentro de cada sucursal — proteger divisiones por cero
# venta_costo%
ema["venta_costo%"] = 0.0
mask_vct_nonzero = ema["venta_costo_tot"] != 0
ema.loc[mask_vct_nonzero, "venta_costo%"] = (
    (ema.loc[mask_vct_nonzero, "venta_costo_ema"] / ema.loc[mask_vct_nonzero, "venta_costo_tot"])
    .round(4)
)

# cantidad%
ema["cantidad%"] = 0.0
mask_te_nonzero = ema["total_ema"] != 0
ema.loc[mask_te_nonzero, "cantidad%"] = (
    (ema.loc[mask_te_nonzero, "EMA"] / ema.loc[mask_te_nonzero, "total_ema"]).round(4)
)

# 4) Orden y acumulados
# Ordenar por costo dentro de cada sucursal (descendente por venta_costo_ema)
ema = ema.sort_values(["store_name", "venta_costo_ema"], ascending=[True, False])
ema["acumulado_costo"] = ema.groupby("store_name")["venta_costo%"].cumsum().round(4)

# Ordenar por cantidad dentro de cada sucursal (descendente por EMA)
ema = ema.sort_values(["store_name", "EMA"], ascending=[True, False])
ema["acumulado_cantidad"] = ema.groupby("store_name")["cantidad%"].cumsum().round(2)

# Acumulado de desviacion si existe
if "desviacion_ema%" in ema.columns:
    ema["acumulado_desviacion"] = ema.groupby("store_name")["desviacion_ema%"].cumsum().round(2)

# 5) Renombrar columnas (estilo reporte Odoo)
ema_def = ema.rename(
    columns={
        "año": "Año",
        "semana_num": "semana",
        "EMA": "Ema",
        "producto_costo_unitario": "Costo unitario",
        "venta_costo_ema": "Venta costo EMA",
        "venta_costo%": "Venta costo %",
        "acumulado_costo": "Acumulado costo",
        "desviacion_ema%": "Desviacion EMA %",
    }
)

# 6) Desviacion en dinero (si corresponde) — usar nombres nuevos
if "Desviacion EMA %" in ema_def.columns:
    ema_def["Desviacion EMA"] = (ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]).round(1)

# 7) Aplicar reglas de desviacion (coef_ventas -> acumulado_desviacion_p)
reglas = [
    {"rango_ini": 0, "rango_fin": 0.8, "valor": 0},
    {"rango_ini": 0.8, "rango_fin": 1.5, "valor": 0.5},
    {"rango_ini": 1.5, "rango_fin": 3, "valor": 0.8},
    {"rango_ini": 3, "rango_fin": 2000, "valor": 1},
]

def asignar_desviacion(coef, reglas):
    for r in reglas:
        if r["rango_ini"] <= coef <= r["rango_fin"]:
            return r["valor"]
    return 0.0

ema_def["acumulado_desviacion_p"] = ema_def["coef_ventas"].fillna(0).apply(lambda x: asignar_desviacion(x, reglas))

# 8) Limpiar columnas
if "Unnamed: 0" in ema_def.columns:
    ema_def = ema_def.drop(columns=["Unnamed: 0"])

# 9) Reglas acumulado costo combinado (con protección cuando la columna max es 0)
def check_div(a, b):
    try:
        return a / b if b != 0 else 0
    except Exception:
        return 0

conf_porcentajes_impacto = {
    "cantidad": 0.5,
    "costo": 0.3,
    "desviacion": 0.2,
    "cantidad_mayor": 0.7,
    "costo_mayor": 0.3,
}

conf_acumulado_cantidad = 80
conf_acumulado_cantidad_desviacion = check_div(conf_acumulado_cantidad, 100)

def calcular_combinado(row):
    acumulado_cantidad_final = round(row["acumulado_cantidad"], 2)
    acumulado_costo_final = round(row["Acumulado costo"], 2)
    acumulado_desviacion_p_final = round(row["acumulado_desviacion_p"], 2)

    if acumulado_cantidad_final >= conf_acumulado_cantidad_desviacion:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad_mayor"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo_mayor"]
        )
    else:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo"]
            + acumulado_desviacion_p_final * conf_porcentajes_impacto["desviacion"]
        )

ema_def["acumulado_combinado"] = ema_def.apply(calcular_combinado, axis=1).round(4)

# Normalizar por grupo: proteger cuando max==0 (entonces dejar 0)
def safe_normalize(x):
    mx = x.max()
    if mx == 0 or np.isnan(mx):
        return x * 0.0
    return (x / mx)

ema_def["acumulado_combinado"] = (
    ema_def.groupby("store_name")["acumulado_combinado"].transform(lambda x: safe_normalize(x)).round(4)
)

# 10) Clasificación Pareto (primero calcular, después aplicar la excepción EMA==0)
def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.8:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

ema_def["Clasificacion"] = ema_def["acumulado_combinado"].apply(clasificar_pareto)

# ---- REGLA FINAL E IMPRESCINDIBLE: si Ema == 0 -> siempre "C" ----
ema_def.loc[ema_def["Ema"].fillna(0) == 0, "Clasificacion"] = "C"



In [5]:
ema_def[(ema_def["store_name"]=="SUCURSAL BUCARAMANGA") & (ema_def["product_ref"]=="BAB02013125")].tail(5)

,store_name,product_ref,Año,Semana,semana,num_facturas,Ema,demanda_ajustada,Desviacion EMA %,coef_ventas,...,total_ema,Venta costo %,cantidad%,Acumulado costo,acumulado_cantidad,acumulado_desviacion,Desviacion EMA,acumulado_desviacion_p,acumulado_combinado,Clasificacion
18730,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-10-21 00:00:00,43,0.0,0.0,0.0,0.0,0.0,...,57131.259931,0.0,0.0,0.6758,0.67,4.007311e+08,0.0,0.0,0.7294,C
18731,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-10-28 00:00:00,44,0.0,0.0,0.0,0.0,0.0,...,57131.259931,0.0,0.0,0.6758,0.67,4.007311e+08,0.0,0.0,0.7294,C
18732,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-11-04 00:00:00,45,0.0,0.0,0.0,0.0,0.0,...,57131.259931,0.0,0.0,0.6758,0.67,4.007311e+08,0.0,0.0,0.7294,C
18733,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-11-11 00:00:00,46,0.0,0.0,0.0,0.0,0.0,...,57131.259931,0.0,0.0,0.6758,0.67,4.007311e+08,0.0,0.0,0.7294,C
18734,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,2025-11-18 00:00:00,47,0.0,0.0,0.0,0.0,0.0,...,57131.259931,0.0,0.0,0.6758,0.67,4.007311e+08,0.0,0.0,0.7294,C


In [6]:
ema_ord = ema_def[["store_name","product_ref","Año","semana","Ema",
"Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo","Desviacion EMA",
"Desviacion EMA %","acumulado_cantidad","coef_ventas","acumulado_desviacion","acumulado_desviacion_p","acumulado_combinado","Clasificacion"]]

ema_ord = ema_ord[~(ema_ord["store_name"]=="0")]

In [7]:
ema_ord["acumulado_combinado"].describe()

count    1.451336e+06
mean     7.876470e-01
std      1.093693e-01
min      1.520000e-02
25%      7.294000e-01
50%      7.406000e-01
75%      7.494000e-01
max      1.000000e+00
Name: acumulado_combinado, dtype: float64

In [8]:
ema_ord[(ema_ord["store_name"]=="SUCURSAL BUCARAMANGA") & (ema_ord["product_ref"]=="BAB02013125")].tail(5)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion
18730,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,43,0.0,227670.52,0.0,0.0,0.6758,0.0,0.0,0.67,0.0,4.007311e+08,0.0,0.7294,C
18731,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,44,0.0,227670.52,0.0,0.0,0.6758,0.0,0.0,0.67,0.0,4.007311e+08,0.0,0.7294,C
18732,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,45,0.0,227670.52,0.0,0.0,0.6758,0.0,0.0,0.67,0.0,4.007311e+08,0.0,0.7294,C
18733,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,46,0.0,227670.52,0.0,0.0,0.6758,0.0,0.0,0.67,0.0,4.007311e+08,0.0,0.7294,C
18734,SUCURSAL BUCARAMANGA,BAB02013125,2025.0,47,0.0,227670.52,0.0,0.0,0.6758,0.0,0.0,0.67,0.0,4.007311e+08,0.0,0.7294,C


## NIVEL DE SERVICIO

In [9]:
# Definimos las condiciones para la meta, basadas en la clasificación
condiciones_meta = [
    (ema_ord['Clasificacion'] == 'AAA'),
    (ema_ord['Clasificacion'] == 'A'),
    (ema_ord['Clasificacion'] == 'B'),
    (ema_ord['Clasificacion'] == 'C')
]

# Definimos los valores de la meta que se asignarán a cada clasificación
metas = [0.999, 0.95, 0.90, 0.50]

ema_ord = ema_ord.copy()

# Asignamos la nueva columna 'Meta' usando np.select
ema_ord.loc[:, 'Nivel de servicio'] = np.select(condiciones_meta, metas, default=np.nan)

In [10]:
ema_ord.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv")

bq = ema_ord[ema_ord["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False)

bq.head((5))

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
642474,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,10,72.187584,14058.32,1014836.16,0.0004,0.0762,71038.5,0.07,0.0,1.179775,0.07,0.5,0.1737,AAA,0.999
642496,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,32,64.296791,14058.32,903904.86,0.0003,0.0864,171741.9,0.19,0.0,3.225859,0.26,1.0,0.3179,AAA,0.999
642485,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,21,60.515350,14058.32,850744.16,0.0003,0.0903,153133.9,0.18,0.0,2.953224,0.44,0.8,0.2619,AAA,0.999
642475,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,11,58.750067,14058.32,825927.24,0.0003,0.0927,115629.8,0.14,0.0,1.894737,0.58,0.8,0.2619,AAA,0.999
642497,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,33,57.637433,14058.32,810285.48,0.0003,0.0957,145851.4,0.18,0.0,3.634043,0.76,1.0,0.3221,AAA,0.999


In [11]:
df_mio = ema_ord[["store_name","product_ref","semana","Ema","acumulado_combinado","Clasificacion"]]


In [12]:
conteo = pd.crosstab(df_mio["store_name"], df_mio["Clasificacion"])
conteo

Clasificacion,A,AAA,B,C
store_name,,,,
PRINCIPAL COTA,17258,548,2796,160815
SUCURSAL BARRANQUILLA,14891,821,4650,161055
SUCURSAL BUCARAMANGA,14919,782,3870,161846
SUCURSAL CALI,14942,554,3402,162519
SUCURSAL CALLE 6,17375,840,4651,158551
SUCURSAL MEDELLIN,14610,732,4118,161957
SUCURSAL NORTE,13759,548,3183,163927
SUCURSAL VALLADOLID,14446,593,3311,163067


In [13]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","acumulado_combinado","clasificacion","almacen_id" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)

df_real =df.copy()


In [14]:
df_mio

,store_name,product_ref,semana,Ema,acumulado_combinado,Clasificacion
1333227,PRINCIPAL COTA,DCS10536137,31,24.297362,0.2506,AAA
236105,PRINCIPAL COTA,BCE00606125,32,24.029336,0.2694,AAA
1333238,PRINCIPAL COTA,DCS10536137,42,22.271082,0.2607,AAA
1333228,PRINCIPAL COTA,DCS10536137,32,21.867626,0.2607,AAA
1333239,PRINCIPAL COTA,DCS10536137,43,20.043974,0.2607,AAA
...,...,...,...,...,...,...
1452007,SUCURSAL VALLADOLID,TAP8111,43,0.000000,0.7449,C
1452008,SUCURSAL VALLADOLID,TAP8111,44,0.000000,0.7449,C
1452009,SUCURSAL VALLADOLID,TAP8111,45,0.000000,0.7449,C
1452010,SUCURSAL VALLADOLID,TAP8111,46,0.000000,0.7449,C


In [15]:
df_real = df_real[["store_name","product_name","semana","ema","acumulado_combinado","clasificacion"]]

In [16]:
df_real

,store_name,product_name,semana,ema,acumulado_combinado,clasificacion
0,SUCURSAL BARRANQUILLA,DAB02570025,46,42.650564,0.220464,AAA
1,SUCURSAL BARRANQUILLA,DAB14570025,46,31.967483,0.305152,AAA
2,SUCURSAL BARRANQUILLA,BCS00035125,46,31.165595,0.236123,AAA
3,SUCURSAL BARRANQUILLA,DAB02772025,46,26.973474,0.297966,AAA
4,SUCURSAL BARRANQUILLA,BLS00037125,46,25.271492,0.236360,AAA
...,...,...,...,...,...,...
43942,SUCURSAL NORTE,DAE05466003,46,0.000000,1.000000,C
43943,SUCURSAL NORTE,DAE05094003,46,0.000000,1.000000,C
43944,SUCURSAL NORTE,DAE06511003,46,0.000000,1.000000,C
43945,SUCURSAL NORTE,DAE06627002,46,0.000000,1.000000,C
